In [85]:
import arcpy

In [86]:
# Set paths

arcpy.env.overwriteOutput = True 

datafolder = r"C:\Users\mil472\Documents\ArcGIS\Packages\BostonPuzzle_3615cc\p20\bostonpuzzle.gdb"

storagefolder = r"C:\Users\mil472\Documents\ArcGIS\Packages\BostonPuzzle_3615cc\p20\bostonpuzzle.gdb"

arcpy.env.workspace = storagefolder 

if arcpy.CheckExtension("spatial") == "Available":
    arcpy.CheckOutExtension("spatial")
else: print("extension is not available")

In [87]:
# Create variables

boston_boundaries = datafolder + r"BostonBoundary"
lidar = datafolder + r"\lidar_Clip"
T_stops = datafolder + r"\MBTA_nodes"
T_lines = datafolder + r"\subway_lines"
sprays = datafolder + r"\Tot_Sprays"
trash = datafolder + r"\TrashDay"
wifi = datafolder + r"\Wicked_Free_WiFi_spots"

In [88]:
# CLUE: The site can put their trash out Sunday night for Monday pickup.

# Include areas where trash day is one of two or three days, of which Mondaty is included. 
where_clause = '\"TRASHDAY\" LIKE \'M%\''

selection = arcpy.SelectLayerByAttribute_management(trash, "NEW_SELECTION", where_clause)
arcpy.CopyFeatures_management(selection, "Trash_Mon")

<Result 'C:\\Users\\mil472\\Documents\\ArcGIS\\Packages\\BostonPuzzle_3615cc\\p20\\bostonpuzzle.gdb\\Trash_Mon'>

In [89]:
# CLUE: The site is in a census block group that contains two wifi hubs.

# Join wifi spots to block group shapefile

arcpy.SpatialJoin_analysis("Trash_Mon", wifi, "wifi_join")

# Select from the joined layer those ssites that contain two wifi hubs

where_clause_2 = '\"Join_Count\" = 2'

selection_2 = arcpy.SelectLayerByAttribute_management("wifi_join", "NEW_SELECTION", where_clause_2)
arcpy.CopyFeatures_management(selection_2, "trash_wifi")

<Result 'C:\\Users\\mil472\\Documents\\ArcGIS\\Packages\\BostonPuzzle_3615cc\\p20\\bostonpuzzle.gdb\\trash_wifi'>

In [96]:
# CLUE: The site is approximately 285 meters from the nearest tot spray and a T stop is even closer.

# Create spray buffer

arcpy.Buffer_analysis(sprays, "spray_buffer_285_m", "285 meters", "FULL", "ROUND", "NONE")

# Find centroids for spray buffer

arcpy.FeatureToPoint_management("spray_buffer_285_m", "buffer_centroids", "CENTROID")

# Select By Location for T Stops

selection_3 = arcpy.SelectLayerByLocation_management("trash_wifi", "WITHIN_A_DISTANCE", T_stops, "285 meters")
arcpy.CopyFeatures_management(selection_3, "t_stop_285")

# Find intersection of spray buffer centroids and selected polygons (those within 285 meters of subway stop)

selection_4 = arcpy.SelectLayerByLocation_management("t_stop_285", "INTERSECT", "buffer_centroids")
arcpy.CopyFeatures_management(selection_4, "t_stop_spray_select")


<Result 'C:\\Users\\mil472\\Documents\\ArcGIS\\Packages\\BostonPuzzle_3615cc\\p20\\bostonpuzzle.gdb\\t_stop_spray_select'>

In [ ]:
# CLUE: The building itself has residents and visitors from all over the world.

# What is the Boston Marriot in Long Wharf?